    

---

# partie Pandas _(3h30)_


---



# Importation et pré-traitement d'un dataset
---



*Le jeu de données à analyser se trouve à l'adresse suivante : (https://divvy-tripdata.s3.amazonaws.com/index.html). Il y a un fichier par mois, pour un total de 12 fichiers. Chaque fichier contient 13 colonnes avec des types de données variés. Nous allons fusionner les fichiers en un seul et le nommer 'combined_data'.*

In [ ]:
# Importation des bibliothèques
import numpy as np
import pandas as pd
import datetime
import requests
import zipfile
import io
import os
from math import radians, sin, cos, sqrt, atan2

In [ ]:
####### Importation des datasets depuis le serveur AWS dédiéc (A EXECUTER UNE SEULE FOIS !!!!)

# Liste des noms de fichiers
file_names = [
    "202101-divvy-tripdata.zip",
    "202102-divvy-tripdata.zip",
    "202103-divvy-tripdata.zip",
    "202104-divvy-tripdata.zip",
    "202105-divvy-tripdata.zip",
    "202106-divvy-tripdata.zip",
    "202107-divvy-tripdata.zip",
    "202108-divvy-tripdata.zip",
    "202109-divvy-tripdata.zip",
    "202110-divvy-tripdata.zip",
    "202111-divvy-tripdata.zip",
    "202112-divvy-tripdata.zip"
]

# URL de base
base_url = "https://divvy-tripdata.s3.amazonaws.com/"

# Télécharger chaque fichier
for file_name in file_names:

    url = base_url + file_name
    response = requests.get(url)

    # Extraire le fichier Zip dans le dossier local "wild_divvy_data"
    with zipfile.ZipFile(io.BytesIO(response.content)) as the_zip:
      the_zip.extractall("wild_divvy_data")


In [ ]:
folder_path = 'wild_divvy_data'

# Obtenir une liste de tous les fichiers CSV dans le dossier local
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialiser un DataFrame vide pour stocker les données combinées
combined_data = pd.DataFrame()

# Parcourir chaque fichier CSV et concaténer ses données au DataFrame combiné
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    combined_data = pd.concat([combined_data, data], ignore_index=True)

# Afficher les données combinées
combined_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C809ED75D6160B2A,electric_bike,2021-05-30 11:58:15,2021-05-30 12:10:39,NaN,NaN,NaN,NaN,41.900000,-87.630000,41.890000,-87.610000,casual
1,DD59FDCE0ACACAF3,electric_bike,2021-05-30 11:29:14,2021-05-30 12:14:09,NaN,NaN,NaN,NaN,41.880000,-87.620000,41.790000,-87.580000,casual
2,0AB83CB88C43EFC2,electric_bike,2021-05-30 14:24:01,2021-05-30 14:25:13,NaN,NaN,NaN,NaN,41.920000,-87.700000,41.920000,-87.700000,casual
3,7881AC6D39110C60,electric_bike,2021-05-30 14:25:51,2021-05-30 14:41:04,NaN,NaN,NaN,NaN,41.920000,-87.700000,41.940000,-87.690000,casual
4,853FA701B4582BAF,electric_bike,2021-05-30 18:15:39,2021-05-30 18:22:32,NaN,NaN,NaN,NaN,41.940000,-87.690000,41.940000,-87.700000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595058,FA66BCAB0D73DDC2,classic_bike,2021-09-22 15:46:57,2021-09-22 16:01:15,Ellis Ave & 83rd St,584,Stony Island Ave & 75th St,KA1503000019,41.744123,-87.599034,41.758670,-87.586883,casual
5595059,1D44DEFB5D36CA04,classic_bike,2021-09-25 16:25:23,2021-09-25 16:40:29,Ellis Ave & 60th St,KA1503000014,Shore Dr & 55th St,TA1308000009,41.785097,-87.601073,41.795212,-87.580715,casual
5595060,6A346EA57FC23C45,classic_bike,2021-09-25 16:26:05,2021-09-25 16:40:30,Ellis Ave & 60th St,KA1503000014,Shore Dr & 55th St,TA1308000009,41.785097,-87.601073,41.795212,-87.580715,casual
5595061,49360AFD771100A6,classic_bike,2021-09-15 17:57:48,2021-09-15 18:24:06,Ellis Ave & 60th St,KA1503000014,Shore Dr & 55th St,TA1308000009,41.785097,-87.601073,41.795212,-87.580715,casual


Pré-traitement
---



1. Afficher les types des colonnes

In [ ]:
# observation des types des différentes colonnes
print(combined_data.dtypes)

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


2. Afficher le nombre de valeurs manquantes par colonne

In [ ]:
# observation des valeurs manquantes par colonne
print(combined_data.isna().sum())

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    690809
start_station_id      690806
end_station_name      739170
end_station_id        739170
start_lat                  0
start_lng                  0
end_lat                 4771
end_lng                 4771
member_casual              0
dtype: int64


3. Identifier et compter les lignes dupliquées pour toutes les colonnes

In [ ]:
# observation des doublons
combined_data.duplicated().sum()

0

## Traitement

*Commentaire :*

4. Créer une nouvelle dataframe en supprimant tous les trajets avec des valeurs manquantes et afficher le nombre de lignes et de colonnes de la nouvelle dataframe

In [ ]:
# copie du dataframe
data_copy = combined_data.copy()


In [ ]:
# suppression des valeurs manquantes
data_copy = data_copy.dropna().reset_index(drop=True)

In [ ]:
# check nombre de lignes et colonnes
data_copy.shape

(4588302, 13)

## Colonnes supplémentaires et transformation des données

Nous ajouterons la distance parcourue en kilomètres à partir des latitudes et longitudes de départ et d'arrivée données. Nous utiliserons la formule de Haversine.

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
# Rayon de la Terre en kilomètres
    R = 6371.0

    # Convertir la latitude et la longitude de degrés en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculer les différences entre les latitudes et les longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Formule de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculer la distance
    distance = R * c

    return distance


5. Ajouter une nouvelle colonne "distance_travelled_km" en appliquant la fonction haversine sur tous les trajets.

> On rappelle le prototype de la fonction haversine(lat1, lon1, lat2, lon2)



In [ ]:
data_copy["distance_travelled_km"] = data_copy.apply( lambda row: haversine(row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]), axis=1 )


In [ ]:
data_copy.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km
0,0F3AE375DEC608D9,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000
1,43E916C72D4C77B3,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617
2,2AAB47362E76AC7F,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267
3,344011882FBBB8BD,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000
4,238E58CAB07FA2A8,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905


6. Analyser les statistiques concernant la colonne "distance_travelled_km".

Que remarquez vous?

In [ ]:
# vérification d'une ligne du corriger pour voir une valeur de la colonne ajouté
data_copy.loc[data_copy["ride_id"] == "89E7AA6C29227EFF"]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km
3924156,89E7AA6C29227EFF,classic_bike,2021-02-12 16:14:56,2021-02-12 16:21:43,Glenwood Ave & Touhy Ave,525,Sheridan Rd & Columbia Ave,660,42.012701,-87.666058,42.004583,-87.661406,member,0.981104


In [ ]:
# la distance ne nous donne que la distance entre les bornes et non la distances parcourue
print(data_copy["distance_travelled_km"].describe())

count    4.588302e+06
mean     2.128845e+00
std      1.879802e+00
min      0.000000e+00
25%      9.038498e-01
50%      1.619725e+00
75%      2.814214e+00
max      3.380018e+01
Name: distance_travelled_km, dtype: float64


*Réponse :*

7. Pour remédier à ce problèm, ajouter une nouvelle colonne "ride_duration_s" pour calculer la période du trajet en secondes

In [ ]:
# Conversion en format date

data_copy["started_at"] = pd.to_datetime(data_copy["started_at"])
data_copy["ended_at"] = pd.to_datetime(data_copy["ended_at"])


In [ ]:
# Création colonne started_at_seconde
data_copy["started_at_seconds"] = data_copy["started_at"].apply(lambda x: x.timestamp())

In [ ]:
# Création colonne ended_at_seconds
data_copy["ended_at_seconds"] = data_copy["ended_at"].apply(lambda x: x.timestamp())

In [ ]:
# création de la différence des deux date fin - debut pour la durée
data_copy["ride_duration_s"] = data_copy["ended_at_seconds"] - data_copy["started_at_seconds"]
data_copy.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,started_at_seconds,ended_at_seconds,ride_duration_s
0,0F3AE375DEC608D9,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,1.619944e+09,1.619947e+09,3567.0
1,43E916C72D4C77B3,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617,1.619907e+09,1.619909e+09,2132.0
2,2AAB47362E76AC7F,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267,1.622422e+09,1.622423e+09,404.0
3,344011882FBBB8BD,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000,1.622447e+09,1.622450e+09,3517.0
4,238E58CAB07FA2A8,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905,1.619975e+09,1.619975e+09,71.0


8. Analyser les statistiques concernant la colonne "ride_duration_s".

Que remarquez vous?

In [ ]:
data_copy["ride_duration_s"].describe()

,ride_duration_s
count,4.588302e+06
mean,1.308638e+03
std,1.106550e+04
min,-3.354000e+03
25%,4.170000e+02
50%,7.320000e+02
75%,1.327000e+03
max,3.356649e+06


In [ ]:
data_copy["ride_duration_formatted"] = pd.to_timedelta(data_copy["ride_duration_s"], unit="s")


In [ ]:
data_copy["ride_duration_formatted"].describe()

,ride_duration_formatted
count,4588302
mean,0 days 00:21:48.637676421
std,0 days 03:04:25.503842186
min,-1 days +23:04:06
25%,0 days 00:06:57
50%,0 days 00:12:12
75%,0 days 00:22:07
max,38 days 20:24:09


*Réponse :*
- Le min est négatif (-3 354 000 secondes), ce qui est illogique pour une durée (une durée ne peut pas être négative). Cela indique probablement des erreurs dans les données.
- Le max est très élevé (3 356 649 secondes, soit environ 39 jours "38 days 20:24:09"), ce qui semble également anormal pour une durée de trajet. ( le vélo n'a pas été rendu, et a été rendu au bout de 39 jours ou anomalie de donnée)

9. Supprimer les trajets de moins d'une minute

In [ ]:
data_copy = data_copy.loc[data_copy["ride_duration_s"] >= 60]

In [ ]:
data_copy["ride_duration_formatted"].describe()

,ride_duration_formatted
count,4528933
mean,0 days 00:22:05.487672041
std,0 days 03:05:36.807784340
min,0 days 00:01:00
25%,0 days 00:07:07
50%,0 days 00:12:23
75%,0 days 00:22:19
max,38 days 20:24:09


In [ ]:
data_copy.shape

(4528933, 18)

10. Créer une colonne "speed_mph" afin d'analyser la vitesse en calculant la vitesse du km/h vers mph.



In [ ]:
data_copy["speed_mph"] =  (data_copy["distance_travelled_km"] * 0.621371) / (data_copy["ride_duration_s"] / 3600)


In [ ]:
data_copy["speed_kmH"] =  data_copy["distance_travelled_km"] / (data_copy["ride_duration_s"] / 3600)

In [ ]:
data_copy.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,started_at_seconds,ended_at_seconds,ride_duration_s,ride_duration_formatted,speed_mph,speed_kmH
0,0F3AE375DEC608D9,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,1.619944e+09,1.619947e+09,3567.0,0 days 00:59:27,0.000000,0.000000
1,43E916C72D4C77B3,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617,1.619907e+09,1.619909e+09,2132.0,0 days 00:35:32,0.031074,0.050009
2,2AAB47362E76AC7F,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267,1.622422e+09,1.622423e+09,404.0,0 days 00:06:44,10.643532,17.129109
3,344011882FBBB8BD,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000,1.622447e+09,1.622450e+09,3517.0,0 days 00:58:37,0.000000,0.000000
4,238E58CAB07FA2A8,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905,1.619975e+09,1.619975e+09,71.0,0 days 00:01:11,0.028517,0.045894


In [ ]:
data_copy["speed_mph"].describe()

,speed_mph
count,4.528933e+06
mean,5.790966e+00
std,2.945404e+00
min,0.000000e+00
25%,4.122913e+00
50%,6.096761e+00
75%,7.738143e+00
max,4.223214e+01


11. Supprimer les données avec une vitesse supérieure à 45 km/h (~28 mph, vitesse maximale des vélos aux États-Unis).

In [ ]:
data_copy["speed_kmH"].describe()

,speed_kmH
count,4.528933e+06
mean,9.319658e+00
std,4.740170e+00
min,0.000000e+00
25%,6.635187e+00
50%,9.811789e+00
75%,1.245334e+01
max,6.796607e+01


In [ ]:
data_copy = data_copy.loc[data_copy["speed_kmH"] < 45]

In [ ]:
data_copy.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'distance_travelled_km', 'started_at_seconds',
       'ended_at_seconds', 'ride_duration_s', 'ride_duration_formatted',
       'speed_mph', 'speed_kmH'],
      dtype='object')

In [ ]:
data_copy = data_copy.drop(['started_at_seconds', 'ended_at_seconds', 'ride_duration_formatted', 'speed_kmH'], axis=1)

In [ ]:
data_copy.shape

(4528930, 16)

12. Supprimer les colonnes ("ride_id" et "speed_kph") pour un chargement plus rapide

In [ ]:
data_copy = data_copy.drop(['ride_id'], axis=1)

13. * Supprimer les espaces de fin pour les colonnes de type chaîne de caractères.

  * Il y a des données qui sont des trajets de test (avec 'test' inclus dans le nom de la station et l'identifiant de la station). Nous supprimerons également ces données.

In [ ]:
# supprression des espaces de début et fin avec strip() en itérant pour automatisé
for col in data_copy.select_dtypes(include=['object']).columns:
  data_copy[col] = data_copy[col].str.strip()

In [ ]:
# supprression des lignes contenant le mot test dans toutes les colonnes de type objet
for col in data_copy.select_dtypes(include=['object']).columns:
  data_copy = data_copy[~data_copy[col].str.contains('test', case=False)]

In [ ]:
data_copy.shape

(4528312, 15)

In [ ]:
data_copy.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'member_casual',
       'distance_travelled_km', 'ride_duration_s', 'speed_mph'],
      dtype='object')

14. Ajouter les catégories suivantes pour découvrir les motifs basés sur le temps :

* day_of_week : Lundi, Mardi, Mercredi, etc.
* day_type : Jour de semaine ou Week-end
* month : Janvier, Février, Mars, etc.
* season : Hiver, Printemps, Été, Automne

    **Créer une fonction pour day_type**

    **Créer une fonction pour season**

In [ ]:
# Traitement 'day_of_week' et 'month'
# Extraction du jour et du mois
data_copy['day_of_week'] = data_copy['started_at'].dt.day_name()
data_copy['month'] = data_copy['started_at'].dt.month_name()
data_copy

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,speed_mph,day_of_week,month
0,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,3567.0,0.000000,Sunday,May
1,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617,2132.0,0.031074,Saturday,May
2,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267,404.0,10.643532,Monday,May
3,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000,3517.0,0.000000,Monday,May
4,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905,71.0,0.028517,Sunday,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588297,classic_bike,2021-09-22 15:46:57,2021-09-22 16:01:15,Ellis Ave & 83rd St,584,Stony Island Ave & 75th St,KA1503000019,41.744123,-87.599034,41.758670,-87.586883,casual,1.905896,858.0,4.968959,Wednesday,September
4588298,classic_bike,2021-09-25 16:25:23,2021-09-25 16:40:29,Ellis Ave & 60th St,KA1503000014,Shore Dr & 55th St,TA1308000009,41.785097,-87.601073,41.795212,-87.580715,casual,2.028199,906.0,5.007672,Saturday,September
4588299,classic_bike,2021-09-25 16:26:05,2021-09-25 16:40:30,Ellis Ave & 60th St,KA1503000014,Shore Dr & 55th St,TA1308000009,41.785097,-87.601073,41.795212,-87.580715,casual,2.028199,865.0,5.245029,Saturday,September
4588300,classic_bike,2021-09-15 17:57:48,2021-09-15 18:24:06,Ellis Ave & 60th St,KA1503000014,Shore Dr & 55th St,TA1308000009,41.785097,-87.601073,41.795212,-87.580715,casual,2.028199,1578.0,2.875127,Wednesday,September


In [ ]:
#Fonction pour 'day_type'
jour_weekend = ["Saturday", "Sunday"]

def extract_name_week(jour):
  if jour in jour_weekend:
    return "Weekend"
  else:
    return "Weekday"


In [ ]:
#Application de la fonction pour 'day_type'
data_copy['day_type'] = data_copy['day_of_week'].apply(extract_name_week)
data_copy.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,speed_mph,day_of_week,month,day_type
0,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,3567.0,0.000000,Sunday,May,Weekend
1,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617,2132.0,0.031074,Saturday,May,Weekend
2,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267,404.0,10.643532,Monday,May,Weekday
3,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000,3517.0,0.000000,Monday,May,Weekday
4,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905,71.0,0.028517,Sunday,May,Weekend


In [ ]:
#Fonction pour 'season'
month_saison = {
    "Winter" : ["December", "January", "February"],
    "Spring" : ["March", "April", "May"],
    "Summer" : ["June", "July", "August"],
    "Autumn" : ["June", "July", "August"]
}

def extract_saison(mois):
    for saison, mois_liste in month_saison.items():
        if mois in mois_liste:
            return saison
    return "Mois invalide"

In [ ]:
#Application de la fonction pour 'season'
data_copy['season'] = data_copy['month'].apply(extract_saison)
data_copy.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,speed_mph,day_of_week,month,day_type,season
0,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,3567.0,0.000000,Sunday,May,Weekend,Spring
1,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617,2132.0,0.031074,Saturday,May,Weekend,Spring
2,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267,404.0,10.643532,Monday,May,Weekday,Spring
3,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000,3517.0,0.000000,Monday,May,Weekday,Spring
4,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905,71.0,0.028517,Sunday,May,Weekend,Spring


15. Ajouter la colonne 'route_type' (même 'start_station_name' et 'end_station_name') pour classifier les trajets comme trajet aller-retour ou trajet aller simple.

In [ ]:
#Fonction for 'route_type'
def route_type(station_aller, station_retour):
  if station_aller == station_retour:
    return "Return trip"
  else:
    return "One-way trip"

In [ ]:
#Application 'route_type'
data_copy["route_type"] = data_copy.apply(lambda row: route_type(row["start_station_name"], row["end_station_name"]), axis=1)
data_copy.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,speed_mph,day_of_week,month,day_type,season,route_type
0,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,3567.0,0.000000,Sunday,May,Weekend,Spring,Return trip
1,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,-87.623852,casual,0.029617,2132.0,0.031074,Saturday,May,Weekend,Spring,Return trip
2,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,-87.667394,casual,1.922267,404.0,10.643532,Monday,May,Weekday,Spring,One-way trip
3,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,-87.677432,casual,0.000000,3517.0,0.000000,Monday,May,Weekday,Spring,Return trip
4,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,-87.623726,casual,0.000905,71.0,0.028517,Sunday,May,Weekend,Spring,Return trip


16. Ajouter la colonne "ride_duration_min" qui représente la durée du trajet en minutes.

In [ ]:
data_copy["ride_duration_min"] = data_copy["ride_duration_s"] / 60
data_copy.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,...,member_casual,distance_travelled_km,ride_duration_s,speed_mph,day_of_week,month,day_type,season,route_type,ride_duration_min
0,classic_bike,2021-05-02 08:21:19,2021-05-02 09:20:46,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,...,casual,0.000000,3567.0,0.000000,Sunday,May,Weekend,Spring,Return trip,59.450000
1,electric_bike,2021-05-01 22:15:55,2021-05-01 22:51:27,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.901037,-87.623789,41.900775,...,casual,0.029617,2132.0,0.031074,Saturday,May,Weekend,Spring,Return trip,35.533333
2,electric_bike,2021-05-31 00:50:15,2021-05-31 00:56:59,Clark St & Grace St,TA1307000127,Clark St & Leland Ave,TA1309000014,41.950929,-87.659147,41.967092,...,casual,1.922267,404.0,10.643532,Monday,May,Weekday,Spring,One-way trip,6.733333
3,classic_bike,2021-05-31 07:35:08,2021-05-31 08:33:45,Sheridan Rd & Noyes St (NU),604,Sheridan Rd & Noyes St (NU),604,42.058239,-87.677432,42.058239,...,casual,0.000000,3517.0,0.000000,Monday,May,Weekday,Spring,Return trip,58.616667
4,electric_bike,2021-05-02 16:55:59,2021-05-02 16:57:10,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900995,-87.623736,41.900992,...,casual,0.000905,71.0,0.028517,Sunday,May,Weekend,Spring,Return trip,1.183333


17. Renommer tes colonnes pour qu'elles soient pretes pour la dataViz

In [ ]:
data_copy.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'member_casual',
       'distance_travelled_km', 'ride_duration_s', 'speed_mph', 'day_of_week',
       'month', 'day_type', 'season', 'route_type', 'ride_duration_min'],
      dtype='object')

In [ ]:
#  Sélectionne uniquement les colonnes spécifiées et dans le meme ordre
columns = ['rideable_type', 'member_casual', 'started_at', 'ended_at', 'day_of_week', 'day_type', 'month',
           'season', 'start_station_name', 'end_station_name', 'route_type', 'start_lat', 'start_lng',
           'end_lat', 'end_lng', 'distance_travelled_km', 'ride_duration_s', 'ride_duration_min']

data_copy = data_copy.loc[:, columns]  # Sélectionne uniquement les colonnes spécifiées
data_copy

,rideable_type,member_casual,started_at,ended_at,day_of_week,day_type,month,season,start_station_name,end_station_name,route_type,start_lat,start_lng,end_lat,end_lng,distance_travelled_km,ride_duration_s,ride_duration_min
0,classic_bike,casual,2021-05-02 08:21:19,2021-05-02 09:20:46,Sunday,Weekend,May,Spring,Michigan Ave & Oak St,Michigan Ave & Oak St,Return trip,41.900960,-87.623777,41.900960,-87.623777,0.000000,3567.0,59.450000
1,electric_bike,casual,2021-05-01 22:15:55,2021-05-01 22:51:27,Saturday,Weekend,May,Spring,Michigan Ave & Oak St,Michigan Ave & Oak St,Return trip,41.901037,-87.623789,41.900775,-87.623852,0.029617,2132.0,35.533333
2,electric_bike,casual,2021-05-31 00:50:15,2021-05-31 00:56:59,Monday,Weekday,May,Spring,Clark St & Grace St,Clark St & Leland Ave,One-way trip,41.950929,-87.659147,41.967092,-87.667394,1.922267,404.0,6.733333
3,classic_bike,casual,2021-05-31 07:35:08,2021-05-31 08:33:45,Monday,Weekday,May,Spring,Sheridan Rd & Noyes St (NU),Sheridan Rd & Noyes St (NU),Return trip,42.058239,-87.677432,42.058239,-87.677432,0.000000,3517.0,58.616667
4,electric_bike,casual,2021-05-02 16:55:59,2021-05-02 16:57:10,Sunday,Weekend,May,Spring,Michigan Ave & Oak St,Michigan Ave & Oak St,Return trip,41.900995,-87.623736,41.900992,-87.623726,0.000905,71.0,1.183333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588297,classic_bike,casual,2021-09-22 15:46:57,2021-09-22 16:01:15,Wednesday,Weekday,September,Mois invalide,Ellis Ave & 83rd St,Stony Island Ave & 75th St,One-way trip,41.744123,-87.599034,41.758670,-87.586883,1.905896,858.0,14.300000
4588298,classic_bike,casual,2021-09-25 16:25:23,2021-09-25 16:40:29,Saturday,Weekend,September,Mois invalide,Ellis Ave & 60th St,Shore Dr & 55th St,One-way trip,41.785097,-87.601073,41.795212,-87.580715,2.028199,906.0,15.100000
4588299,classic_bike,casual,2021-09-25 16:26:05,2021-09-25 16:40:30,Saturday,Weekend,September,Mois invalide,Ellis Ave & 60th St,Shore Dr & 55th St,One-way trip,41.785097,-87.601073,41.795212,-87.580715,2.028199,865.0,14.416667
4588300,classic_bike,casual,2021-09-15 17:57:48,2021-09-15 18:24:06,Wednesday,Weekday,September,Mois invalide,Ellis Ave & 60th St,Shore Dr & 55th St,One-way trip,41.785097,-87.601073,41.795212,-87.580715,2.028199,1578.0,26.300000


In [ ]:
data_copy.columns

Index(['rideable_type', 'member_casual', 'started_at', 'ended_at',
       'day_of_week', 'day_type', 'month', 'season', 'start_station_name',
       'end_station_name', 'route_type', 'start_lat', 'start_lng', 'end_lat',
       'end_lng', 'distance_travelled_km', 'ride_duration_s',
       'ride_duration_min'],
      dtype='object')

In [ ]:
# renommer les colonnes comme il faut
data_copy = data_copy.rename(columns= {
    "rideable_type" : "bike_type",
    "member_casual" : "user_type",
    "started_at" : "start_time",
    "ended_at" : "end_time"
})

data_copy

,bike_type,user_type,start_time,end_time,day_of_week,day_type,month,season,start_station_name,end_station_name,route_type,start_lat,start_lng,end_lat,end_lng,distance_travelled_km,ride_duration_s,ride_duration_min
0,classic_bike,casual,2021-05-02 08:21:19,2021-05-02 09:20:46,Sunday,Weekend,May,Spring,Michigan Ave & Oak St,Michigan Ave & Oak St,Return trip,41.900960,-87.623777,41.900960,-87.623777,0.000000,3567.0,59.450000
1,electric_bike,casual,2021-05-01 22:15:55,2021-05-01 22:51:27,Saturday,Weekend,May,Spring,Michigan Ave & Oak St,Michigan Ave & Oak St,Return trip,41.901037,-87.623789,41.900775,-87.623852,0.029617,2132.0,35.533333
2,electric_bike,casual,2021-05-31 00:50:15,2021-05-31 00:56:59,Monday,Weekday,May,Spring,Clark St & Grace St,Clark St & Leland Ave,One-way trip,41.950929,-87.659147,41.967092,-87.667394,1.922267,404.0,6.733333
3,classic_bike,casual,2021-05-31 07:35:08,2021-05-31 08:33:45,Monday,Weekday,May,Spring,Sheridan Rd & Noyes St (NU),Sheridan Rd & Noyes St (NU),Return trip,42.058239,-87.677432,42.058239,-87.677432,0.000000,3517.0,58.616667
4,electric_bike,casual,2021-05-02 16:55:59,2021-05-02 16:57:10,Sunday,Weekend,May,Spring,Michigan Ave & Oak St,Michigan Ave & Oak St,Return trip,41.900995,-87.623736,41.900992,-87.623726,0.000905,71.0,1.183333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588297,classic_bike,casual,2021-09-22 15:46:57,2021-09-22 16:01:15,Wednesday,Weekday,September,Mois invalide,Ellis Ave & 83rd St,Stony Island Ave & 75th St,One-way trip,41.744123,-87.599034,41.758670,-87.586883,1.905896,858.0,14.300000
4588298,classic_bike,casual,2021-09-25 16:25:23,2021-09-25 16:40:29,Saturday,Weekend,September,Mois invalide,Ellis Ave & 60th St,Shore Dr & 55th St,One-way trip,41.785097,-87.601073,41.795212,-87.580715,2.028199,906.0,15.100000
4588299,classic_bike,casual,2021-09-25 16:26:05,2021-09-25 16:40:30,Saturday,Weekend,September,Mois invalide,Ellis Ave & 60th St,Shore Dr & 55th St,One-way trip,41.785097,-87.601073,41.795212,-87.580715,2.028199,865.0,14.416667
4588300,classic_bike,casual,2021-09-15 17:57:48,2021-09-15 18:24:06,Wednesday,Weekday,September,Mois invalide,Ellis Ave & 60th St,Shore Dr & 55th St,One-way trip,41.785097,-87.601073,41.795212,-87.580715,2.028199,1578.0,26.300000


18. Vérifier les caractéristiques des dataframes avant l'export du dataset nettoyé

In [ ]:
# check lignes et colonnes
data_copy.shape

(4528312, 18)

## Extraire le dataframe pour l'analyse

19. Exporter le dataframe pour l'utiliser par la suite dans ton Notebook d'analyse

In [ ]:
#Extract csv file (SI BESOIN)
#from google.colab import files
data_copy.to_csv('df_cyclistic_clean.csv', index=False)
#files.download('df_cyclistic_clean.csv')



---


# Business Case - CDA - Cyclistic (7h)



---


## Contexte

Cyclistic est une entreprise de partage de vélos à Chicago, qui a depuis élargi son offre pour inclure une flotte de 5 824 vélos géolocalisés stationnés à 692 emplacements à travers Chicago.

Les vélos peuvent être déverrouillés à une station et retournés à n'importe quelle autre station du réseau à tout moment.

Les individus achetant des passes pour une seule course ou pour une journée complète sont considérés comme des cyclistes occasionnels, tandis que ceux qui acquièrent des abonnements annuels sont reconnus comme des membres de Cyclistic.



---


## Votre mission

Tu es un analyste de données junior travaillant dans l'équipe des analystes marketing chez Cyclistic.

Le directeur du marketing pense que le succès futur de l'entreprise dépend de l'augmentation du nombre d'abonnements annuels.

Par conséquent, ton équipe souhaite comprendre comment les cyclistes occasionnels et les membres annuels utilisent différemment les vélos Cyclistic.

À partir de ces informations, ton équipe élaborera une nouvelle stratégie marketing pour convertir les cyclistes occasionnels en membres annuels.

Mais avant cela, les dirigeants de Cyclistic doivent approuver tes recommandations, donc il est essentiel d'avoir des analyses de données convaincantes et des visualisations de données professionnelles.

l y a trois questions clés qui doivent être répondues à la fin de l'étude de cas :

* Comment les membres annuels et les cyclistes occasionnels utilisent-ils les vélos Cyclistic différemment ?
* Pourquoi les cyclistes occasionnels achèteraient-ils des abonnements annuels Cyclistic ?
* Comment Cyclistic peut-il utiliser les médias numériques pour influencer les cyclistes occasionnels à devenir membres ?
* Où recommenderas-tu à Cyclistic d'implémenter des nouvelles stations pour rendre le service plus disponible ?



---
##Phase 1 : Collecte, pré-traitement et traitement des données

Dans cette phase, tu dois comprendre le processus de génération et de collecte des données, ainsi que reconnaître les différents types, formats et structures de données pour une utilisation efficace dans l'analyse des données. De plus, tu dois veiller à respecter des pratiques éthiques en matière d'analyse de données et traiter les questions de biais et de crédibilité des données.

Vous allez dans un premier temps explorer le jeu de données, dans le but d'explorer plusieurs possibilités de choix. Une fois que votre choix est fait, lancez-vous dans l'élaboration de votre tableau de bord, dans le but de convaincre un maximum les investisseurs.

**Les données sont disponibles publiquement sur un serveur AWS (https://divvy-tripdata.s3.amazonaws.com/index.html)**

Tu as été chargés de travailler avec une année entière de données, **l'année 2021**, donc il faudra récupérer les fichiers (au format CSV) contenant des données de **Janvier 2021 à Décembre 2021, un fichier pour chaque mois**.

**Pensez à bien explorer les différents fichiers et de pouvoir réfélchir à travailler sur toute l'année. Il faudra aussi penser à éliminer tous les trajets qui causent des problèmes techniques en phase de pré-traitement. N'hésitez pas à ajouter des nouvelles colonnes afin de pouvoir mener à bien vos analyses et visualisations**.




---


## **IMPORTANT**

**P.S : PENSE A UTILISER LE FICHIER CSV CLEAN QUE TU AS PU GENERE A LA FIN DE LA PARTIE PANDAS AU LIEU DE RECOMMENCER LE TRAVAILE A 0. CA TE SERIVRA DE BONNE BASE POUR RAJOUTER QUE LES INFOS DONT T'AS BESOIN POUR CONTINUER LE BUSINESS CASE.**

**Tu peux aussi utiliser le code ci-dessous pour récupérer une bonne base de travail ou si tu n'as pas pu terminer la partie Pandas**

In [ ]:
file_url = "https://storage.googleapis.com/dataset-storage-chicago-cyclist-csv/cyclistic_clean.csv"
chunk_size = 500000  # Nombre de lignes par chunk
chunks = pd.read_csv(file_url, chunksize=chunk_size)

df_list = []
for chunk in chunks: # Ex. filtrer les trajets avec distance > 0
    df_list.append(chunk)

df = pd.concat(df_list, ignore_index=True)
df.head()

,bike_type,user_type,start_time,end_time,day_of_week,day_type,month,season,start_station_name,end_station_name,route_type,start_lat,start_lng,end_lat,end_lng,distance_travelled_km,ride_duration_s,ride_duration_min
0,classic_bike,member,2021-01-24 19:15:38,2021-01-24 19:22:51,Sunday,Weekend,January,Winter,California Ave & Cortez St,Wood St & Augusta Blvd,One-way trip,41.900363,-87.696704,41.899181,-87.672200,2.032303,433.0,7.216667
1,electric_bike,member,2021-01-23 12:57:38,2021-01-23 13:02:10,Saturday,Weekend,January,Winter,California Ave & Cortez St,California Ave & North Ave,One-way trip,41.900406,-87.696733,41.910435,-87.696890,1.115287,272.0,4.533333
2,electric_bike,casual,2021-01-09 15:28:04,2021-01-09 15:37:51,Saturday,Weekend,January,Winter,California Ave & Cortez St,Wood St & Augusta Blvd,One-way trip,41.900374,-87.696688,41.899180,-87.672178,2.032911,587.0,9.783333
3,electric_bike,casual,2021-01-09 15:28:57,2021-01-09 15:37:54,Saturday,Weekend,January,Winter,California Ave & Cortez St,Wood St & Augusta Blvd,One-way trip,41.900379,-87.696716,41.899149,-87.672177,2.035544,537.0,8.950000
4,classic_bike,casual,2021-01-24 15:56:59,2021-01-24 16:07:08,Sunday,Weekend,January,Winter,California Ave & Cortez St,Wood St & Augusta Blvd,One-way trip,41.900363,-87.696704,41.899181,-87.672200,2.032303,609.0,10.150000


In [ ]:
# Exemple d'importation d'un fichier depuis le servers AWS dédié

import requests
import zipfile
import io
import pandas as pd

# Step 1: Télécharger le fichier Zip
url = "https://divvy-tripdata.s3.amazonaws.com/202101-divvy-tripdata.zip"
response = requests.get(url)

# Step 2: Extraire le fichier Zip
with zipfile.ZipFile(io.BytesIO(response.content)) as the_zip:
    the_zip.extractall("divvy_data")

# Step 3: Lire le fichier CSV
csv_file_path = "divvy_data/202101-divvy-tripdata.csv"
df_janvier_2021 = pd.read_csv(csv_file_path)

# Afficher les 5 premières lignes du fichier csv Janvier 2021
df_janvier_2021.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual


In [ ]:
from google.colab import files

# Étape 1 : Téléverser le fichier depuis l'ordinateur
result = files.upload()

In [ ]:
# Étape 2 : Charger le fichier CSV téléversé dans un DataFrame
# Remplacez 'votre_fichier.csv' par le nom exact du fichier téléversé
df_cyclistic = pd.read_csv('')
df_cyclistic.head()



---

##Phase 2 : Analyses et visualisation des données

Pour mettre toutes les chances de votre côté et séduire les investisseurs, vous allez devoir faire les bons choix. Voici quelques questions pour vous guider:
- Quel est le pourcentage des types d'utilisateurs par rapport au total des utilisateurs ?
- Y a-t-il un type de vélo préféré par les différents types d'utilisateurs ?
- Quel type de vélo a la durée de trajet la plus longue entre les utilisateurs ?
- Quelle est la durée moyenne des trajets par type d'utilisateur ?
- Quelle est la distance moyenne parcourue par type d'utilisateur ?
- Quels jours sont les plus actifs pour les utilisateurs ?
- Quels mois ou quelles saisons de l'année les utilisateurs ont tendance à utiliser le service de partage de vélos ?
- Quel type d'utilisateur a le plus grand nombre de trajets aller-retour et de trajets aller simple effectués ?


---


## Phase 3 : Cartographie et recommendation d'implémentation physique

En plus de fournir une analyse complète qui vous permettra d'appuyer vos choix des recommendations à faire à l'entreprise, **prenez soin d'indiquer précisément où est meilleur de s'implanter.**

N'oubliez pas que les investisseurs, qui veulent bien placer leur argent, ne manqueront pas de soulever des points précis, pour s'assurer que votre activité soit rentable à coups sûr.  

**Dans le dataset, vous trouverez les coordonnées des stations de départ et d'arrivées.**

Comme dis plus haut, soyez créatif !   

Ne manquez pas d'étoffer vos arguments, et d'être étincelant le jour J ;)



---


## Livrables attendus

Les investisseurs souhaitent une présentation avec des slides (sous format pdf) et un tableau de bord intéractif (Power BI, Tableau ou autre outil de BI).

La présentation contiendra à minima ces éléments :
- Rappel du contexte et de la problématique
- Arguments
- Analyse exploratoire des données
- Méthodologie, outils et langages utilisés
- Votre tableau de bord contient des graphiques
- Votre tableau de bord contient au moins une visualisation de données interactive
- Votre tableau de bord contient au moins une carte représentant des informations géographiques
- Votre tableau de bord contient au moins un tableau croisé
- L'ensemble des graphiques et visuels doit être lisible par tous (prise en compte des personnes en situation de handicap visuel)
- Réponse aux questions métiers en conclusion :
  1. **Quelles sont les recommandations pour Cyclistic afin d'augmenter leur taux de clientèle, en se concentrant particulièrement sur le ciblage de leurs offres proposées et en exploitant les habitudes d'utilisation des clients ainsi que les préférences pour les vélos?**
  2. **Quel est le meilleur emplacement pour installer de nouvelles stations en se basant sur les habitudes d'utilisation des clients, les saisons et les stations les plus fréquentées ?**


Vous devrez faire **une présentation dans un premier temps sous slides**, qui contient au minimum **le contexte, la problématique, ainsi que la conclusion qui contient vos ARGUMENTS pour retenir l'attention des investisseurs. Aussi, vous utiliserez un dashboard pour convaincre les investisseurs, pour qu'ils puissent visualiser votre analyse et vos arguments.**

**Pour rappel, l'argumentation est l'élément central d'une présentation réalisée par un data analyst. En effet, à vous d'être convaincant, pour que vos arguments soient entendus. Il est donc nécessaire de développer un minimum d'expertise métier sur le sujet, pour être à l'aise.**

Les investisseurs ne sont pas data analyst, mais souhaiterait comprendre la démarche. Il faudra donc s'attacher à expliquer vos choix sans rentrer dans un trop grand niveau technique, autrement dit : vulgariser.

Le juré examinateur, lors de la partie question, peut vous poser des questions sur le code qui est contenu dans votre google colab. Il est donc indispensable de mettre à disposition dans les pièces soumises pour le passage oral, votre travail exploratoire contenu dans un google colab.